In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [3]:
# import libraries
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field

In [4]:
# model
class WeatherResponse(BaseModel):
    """Respond to the user with this"""
    temperature: float = Field(description="The temperature in fahrenheit")
    wind_directon: str = Field(
        description="The direction of the wind in abbreviated form"
    )
    wind_speed: float = Field(description="The speed of the wind in km/h")

In [6]:
model = ChatAnthropic(model="claude-3-5-sonnet-20240620")
model_with_structured_output = model.with_structured_output(WeatherResponse)

In [7]:
response = model_with_structured_output.invoke("The weather in SF is 75 degrees and sunny in SF, with 3 mph winds in the South-East direction")

In [8]:
print(type(response))

<class '__main__.WeatherResponse'>


In [9]:
print(response)

temperature=75.0 wind_directon='SE' wind_speed=4.83


In [10]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Instantiate the LLM model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

class Trivia(BaseModel):
    question: str = Field(description="The trivia question")
    answer: str = Field(description="The correct answer to the trivia question")

# Define the prompt template
prompt = ChatPromptTemplate.from_template(
    "Give me a trivia question about {topic}, respond in JSON with `question` and `answer` keys"
)

# Create a structured LLM using the `with_structured_output` method
structured_llm = model.with_structured_output(Trivia, method="json_mode")

# Chain the prompt and structured LLM using the pipe operator
trivia_chain = prompt | structured_llm

In [11]:
result = trivia_chain.invoke({"topic": "space"})

In [12]:
print(type(result))

<class '__main__.Trivia'>


In [13]:
print(result)

question='What is the largest planet in our solar system?' answer='Jupiter'


In [15]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class Book(BaseModel):
    """Information about a book."""

    title: str = Field(..., description="The title of the book")
    pages: int = Field(
        ..., description="The number of pages in the book."
    )


class Library(BaseModel):
    """Details about all books in a collection."""

    books: List[Book]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=Library)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Query
query = "Please provide details about the books 'The Great Gatsby' with 208 pages and 'To Kill a Mockingbird' with 384 pages."

# Print the prompt and output schema
print(prompt.invoke(query).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Details about all books in a collection.", "properties": {"books": {"title": "Books", "type": "array", "items": {"$ref": "#/definitions/Book"}}}, "required": ["books"], "definitions": {"Book": {"title": "Book", "description": "Information about a book.", "type": "object", "properties": {"title": {"title": "Title", "description": "The title of the book", "type": "string"}, "pages": {"title": "Pages", "description": "The number of pages in the book.", "type": "integer

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)
chain = prompt | llm | parser

In [17]:
result = chain.invoke({"query": query})

In [18]:
result

Library(books=[Book(title='The Great Gatsby', pages=208), Book(title='To Kill a Mockingbird', pages=384)])